In [ ]:
# Import useful mathematical libraries
import numpy as np
import pandas as pd
import glob

# Import useful Machine learning libraries
import gensim

# Import utility files
from utils import read_df, save_object, load_object

In [ ]:
# set the model name
model_name = "example"

In [ ]:
# set the folder for saved objects
import os
directories = ['objects', 'objects/subreddit_post_analysis']
for dirname in directories:
    if not os.path.exists(dirname):
        os.makedirs(dirname)

In [ ]:
# Get the data from the csv files, assumed to be in a directory 'data'
# indexed by name of the author MAKE SURE author is in column index 2 (position 3)
# This saves each csv month into an individual dataframe, to be referenced later
dirname = 'data'
extension = "/*.csv"
file_count = 0
df = pd.DataFrame()
df_list =[]
fnames = glob.glob(dirname + extension)
for fname in fnames:
    df = pd.read_csv(fname, header=0, index_col = 2)
    # save the data frame of posts
    save_object(df, 'objects/subreddit_post_analysis/', model_name + "-posts_dataframe-" + str(file_count))
    file_count += 1

In [ ]:
# create a variable to hold subreddits and their post counts, and the total number of posts
subreddits = []
subreddit_counts = []
sum_posts = 0
sum_deleted_posts = 0
# iterate through saved datafranes
for i in range(file_count):
    df = load_object('objects/subreddit_post_analysis/', model_name + "-posts_dataframe-" + str(i))
    sum_posts += len(df)
    sum_deleted_posts += len(df.loc[['[deleted]']])
    # iterate through each dataframe to analze the percentages of posts
    for j in range(len(df)):
        if not (df.iloc[j].subreddit in subreddits):
            subreddits.append(df.iloc[j].subreddit)
            subreddit_counts.append(1)
        else:
            subreddit_counts[subreddits.index(df.iloc[j].subreddit)] += 1

In [ ]:
# sort the subreddits and counts by highest percentage
subreddit_counts, subreddits = (list(t) for t in zip(*sorted(zip(subreddit_counts, subreddits), reverse=True)))

In [ ]:
# save subreddit_counts and subreddits
save_object(subreddit_counts, 'objects/subreddit_post_analysis/', model_name + "-subreddit_counts")
save_object(subreddits, 'objects/subreddit_post_analysis/', model_name + "-subreddits")

In [ ]:
# print the percentages of posts in each subreddit present
for subreddit in subreddits:
    print(str(subreddit), end=": ")
    print(subreddit_counts[subreddits.index(subreddit)]*100/sum_posts)

print("Total Number of Posts: " + str(sum_posts))
print("Total Number of Posts After Deletion: " + str(sum_posts - sum_deleted_posts))